In [2]:
# Importation des Bibliothèques 
import os
import sys 
sys.path.append('/home/onyxia/work/Projet_Teledec/scripts')
import numpy as np
from osgeo import gdal
from my_function import (
    clip_raster_with_shapefile,
    reproject_and_resample,
    compute_ndvi,
    save_raster)


In [3]:
#  Définition des paramètres 

input_raster_dir = "/home/onyxia/work/data/images"  
shapefile_path = "/home/onyxia/work/data/project/emprise_etude.shp"
mask_foret_path = "/home/onyxia/Projet_Teledec/results/data/img_pretraitees/masque_foret.tif"
output_dir = "/home/onyxia/work/Projet_Teledec"

os.makedirs(output_dir, exist_ok=True)

In [4]:
# Partie 1 : Image multibande
output_multiband_path = os.path.join(output_dir, "Serie_temp_S2_allbands.tif")
bands_to_use = ["2", "3", "4", "5", "6", "7", "8", "8A", "11", "12"]

In [5]:
# Liste des images dans le dossier d'entrée
L_images = sorted(os.listdir(input_raster_dir))
if '.keep' in L_images:
    L_images.remove('.keep')  # Supprime '.keep' si présent

In [6]:
try:
    # Vérifier le chemin de sortie
    output_dir = os.path.dirname(output_multiband_path)
    if not os.path.exists(output_dir):
        raise RuntimeError(f"Le répertoire de sortie n'existe pas : {output_dir}")
    if not os.access(output_dir, os.W_OK):
        raise RuntimeError(f"Permission d'écriture refusée dans le répertoire de sortie : {output_dir}")

    # Ouverture du premier raster qui sert comme une référence
    ref_raster_path = os.path.join(input_raster_dir, L_images[0])
    ref_raster = gdal.Open(ref_raster_path)
    if not ref_raster:
        raise RuntimeError(f"Impossible d'ouvrir le fichier raster de référence : {ref_raster_path}")

    # Vérification de driver GDAL
    driver = gdal.GetDriverByName('GTiff')
    if not driver:
        raise RuntimeError("Le driver GTiff n'est pas disponible.")

    # Création de raster de sortie
    out_raster = driver.Create(
    output_multiband_path,
    ref_raster.RasterXSize,
    ref_raster.RasterYSize,
    len(bands_to_use) * 6,
    gdal.GDT_Byte,
    options=["COMPRESS=LZW", "TILED=YES"] ) # Options pour compresser le raster de sortie 

    # out_raster = driver.Create(output_multiband_path, cols, rows, len(L_images), datatype)
    if not out_raster:
        raise RuntimeError("Impossible de créer le raster de sortie.")
    
    # Ajout de la projection et la géotransformation
    geotransform = ref_raster.GetGeoTransform()
    projection = ref_raster.GetProjection()
    out_raster.SetProjection(projection)
    out_raster.SetGeoTransform(geotransform)

    # Remplissage de raster vide qu'on vient de le crée par les données des bandes 
    band_idx = 1
    for image in L_images:
        raster_path = os.path.join(input_raster_dir, image)
        src_raster = gdal.Open(raster_path)
        if not src_raster:
            raise RuntimeError(f"Impossible d'ouvrir le fichier raster : {raster_path}")
    # Rééchantillonnage du raster source pour qu'il corresponde au raster de sortie
    src_raster = gdal.Warp(
    '',  # Pas besoin d'écrire un fichier temporaire
    src_raster,
    format = 'MEM',  # c'est optionnel pour avoir un  format en mémoire
    width = out_raster.RasterXSize,
    height = out_raster.RasterYSize,
    resampleAlg = gdal.GRA_Bilinear  # Exemple de méthode de rééchantillonnage. Cette méthode on l'a pas vu dans le cours mais ca marche bien 
)
    data = src_raster.GetRasterBand(1).ReadAsArray()
    # Écrire les données
    out_raster.GetRasterBand(band_idx).WriteArray(data)
    band_idx += 1
    src_raster = None

except Exception as e:
    print(f"Erreur : {e}")

finally:
    ref_raster = None
    out_raster = None

print(f"Raster multibande créé avec succès : {output_multiband_path}")  

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Raster multibande créé avec succès : /home/onyxia/work/Projet_Teledec/Serie_temp_S2_allbands.tif


In [ ]:
# Découpage selon l'emprise d'étude
clip_raster_with_shapefile(output_multiband_path, shapefile_path, output_multiband_path)

# Application de masque forêt
with gdal.Open(output_multiband_path) as src, gdal.Open(mask_foret_path) as mask:
    mask_data = mask.ReadAsArray()
    out_image = src.ReadAsArray()
    out_image[:, mask_data == 0] = 0  # Masquage des pixels qui correspondent pas à une classe forêt
    save_raster(out_image, output_multiband_path, output_multiband_path, gdal.GDT_UInt16, 0)